In [ ]:
pip install torch torchvision transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 7.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
import torch
from datasets import load_dataset

# SQuAD veri seti
dataset = load_dataset('squad')


train_size = len(dataset['train'])
validation_size = len(dataset['validation'])

print(f"Train dataset size: {train_size}")
print(f"Validation dataset size: {validation_size}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Train dataset size: 87599
Validation dataset size: 10570


In [ ]:

train_subset = dataset['train'].shuffle(seed=42).select(range(40000))
validation_subset = dataset['validation'].shuffle(seed=42).select(range(4000))

print(f"Train subset size: {len(train_subset)}")
print(f"Validation subset size: {len(validation_subset)}")

Train subset size: 40000
Validation subset size: 4000


In [ ]:
from transformers import BertTokenizerFast, BertForQuestionAnswering


tokenizer = BertTokenizerFast.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')



tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=512,
        truncation="only_second",
        padding="max_length",
        return_offsets_mapping=True,
        return_tensors="pt"
    )


    print(f"Input IDs shape: {inputs['input_ids'].shape}")
    print(f"Attention Mask shape: {inputs['attention_mask'].shape}")
    print(f"Tokenized input IDs: {inputs['input_ids'][0][:10]}")

    offset_mapping = inputs.pop("offset_mapping")
    start_positions = []
    end_positions = []
    for i, offset in enumerate(offset_mapping):
        answer = examples["answers"][i]
        start_char = answer["answer_start"][0]
        end_char = start_char + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        context_start = sequence_ids.index(1)
        context_end = len(sequence_ids) - 1 - sequence_ids[::-1].index(1)

        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            idx_start = context_start
            while idx_start < len(offset) and offset[idx_start][0] <= start_char:
                idx_start += 1
            start_positions.append(idx_start - 1)

            idx_end = context_end
            while idx_end >= 0 and offset[idx_end][1] >= end_char:
                idx_end -= 1
            end_positions.append(idx_end + 1)


    print(f"Start positions: {start_positions[:5]}")
    print(f"End positions: {end_positions[:5]}")

    inputs.update({"start_positions": torch.tensor(start_positions), "end_positions": torch.tensor(end_positions)})
    return inputs


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW
from tqdm import tqdm

tokenized_validation = validation_subset.map(preprocess_function, batched=True, remove_columns=validation_subset.column_names)
validation_dataset = tokenized_validation.with_format("torch")

validation_dataloader = DataLoader(validation_dataset, sampler=SequentialSampler(validation_dataset), batch_size=8)


Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Input IDs shape: torch.Size([1000, 512])
Attention Mask shape: torch.Size([1000, 512])
Tokenized input IDs: tensor([ 101, 1999, 2054, 2095, 2106, 4404, 2034, 5478, 2336, 2000])
Start positions: [42, 129, 160, 42, 53]
End positions: [42, 129, 162, 46, 54]
Input IDs shape: torch.Size([1000, 512])
Attention Mask shape: torch.Size([1000, 512])
Tokenized input IDs: tensor([  101,  2054,  2136,  2018,  1996,  7290, 12482,  1998,  4210,  2412])
Start positions: [118, 19, 99, 60, 13]
End positions: [119, 20, 103, 64, 24]
Input IDs shape: torch.Size([1000, 512])
Attention Mask shape: torch.Size([1000, 512])
Tokenized input IDs: tensor([  101,  2012,  2054,  6466,  2020,  1996,  2967,  1997, 24399,  2275])
Start positions: [96, 24, 63, 16, 85]
End positions: [96, 26, 70, 19, 85]
Input IDs shape: torch.Size([1000, 512])
Attention Mask shape: torch.Size([1000, 512])
Tokenized input IDs: tensor([  101,  2054,  2832,  2079,  8777,  1998,  5290,  2923, 27256,  2015])
Start positions: [29, 52, 77, 31,

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
model.eval()


BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,)

In [ ]:
def get_answer(question, context):
    inputs = tokenizer.encode_plus(question, context, return_tensors='pt', truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    input_ids = inputs["input_ids"].tolist()[0]


    print(f"Encoded input IDs: {input_ids[:10]}")
    print(f"Inputs shapes: {[inputs[k].shape for k in inputs]}")

    outputs = model(**inputs)

    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits


    print(f"Start logits shape: {answer_start_scores.shape}")
    print(f"End logits shape: {answer_end_scores.shape}")
    print(f"Start logits: {answer_start_scores[0][:10]}")
    print(f"End logits: {answer_end_scores[0][:10]}")

    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores) + 1

    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
    return answer


In [ ]:

for i in range(10):
    question = validation_subset[i]["question"]
    context = validation_subset[i]["context"]
    real_answer = validation_subset[i]["answers"]["text"][0]
    predicted_answer = get_answer(question, context)

    print(f"Example {i+1}:")
    print(f"Question: {question}")
    print(f"Context: {context[:200]}...")
    print(f"Real Answer: {real_answer}")
    print(f"Predicted Answer: {predicted_answer}")
    print("="*50)


Encoded input IDs: [101, 1999, 2054, 2095, 2106, 4404, 2034, 5478, 2336, 2000]
Inputs shapes: [torch.Size([1, 148]), torch.Size([1, 148]), torch.Size([1, 148])]
Start logits shape: torch.Size([1, 148])
End logits shape: torch.Size([1, 148])
Start logits: tensor([-6.0500, -2.5139, -2.2918, -7.0159, -8.6057, -6.9928, -8.2870, -7.7616,
        -8.2162, -8.4716], device='cuda:0', grad_fn=<SliceBackward0>)
End logits: tensor([-2.5624, -3.1091, -1.5128, -3.5991, -5.7632, -5.2422, -6.3285, -5.7801,
        -6.5040, -6.8480], device='cuda:0', grad_fn=<SliceBackward0>)
Example 1:
Question: In what year did Massachusetts first require children to be educated in schools?
Context: Private schooling in the United States has been debated by educators, lawmakers and parents, since the beginnings of compulsory education in Massachusetts in 1852. The Supreme Court precedent appears ...
Real Answer: 1852
Predicted Answer: 1852
Encoded input IDs: [101, 2043, 2020, 2358, 21716, 16308, 3603, 1029, 102, 199

In [ ]:

question = "What is the capital of Germany?"
context = "Germany is a country in Central Europe. Its capital is Berlin, which is also the largest city in the country. Berlin is known for its art scene and modern landmarks."


predicted_answer = get_answer(question, context)

print(f"Question: {question}")
print(f"Context: {context}")
print(f"Predicted Answer: {predicted_answer}")


Encoded input IDs: [101, 2054, 2003, 1996, 3007, 1997, 2762, 1029, 102, 2762]
Inputs shapes: [torch.Size([1, 44]), torch.Size([1, 44]), torch.Size([1, 44])]
Start logits shape: torch.Size([1, 44])
End logits shape: torch.Size([1, 44])
Start logits: tensor([-6.6533, -3.5687, -8.0753, -6.7004, -7.1006, -7.6126, -5.8677, -9.0112,
        -6.6532,  0.5760], device='cuda:0', grad_fn=<SliceBackward0>)
End logits: tensor([-1.8279, -2.8440, -6.0870, -6.9146, -5.9055, -6.3068, -3.8081, -5.7387,
        -1.8278, -0.0239], device='cuda:0', grad_fn=<SliceBackward0>)
Question: What is the capital of Germany?
Context: Germany is a country in Central Europe. Its capital is Berlin, which is also the largest city in the country. Berlin is known for its art scene and modern landmarks.
Predicted Answer: berlin


In [ ]:
from sklearn.metrics import accuracy_score

def compute_metrics(preds, labels):
    start_pred, end_pred = preds
    start_labels, end_labels = labels

    start_acc = accuracy_score(start_labels, start_pred)
    end_acc = accuracy_score(end_labels, end_pred)

    return {
        "start_accuracy": start_acc,
        "end_accuracy": end_acc,
    }

model.eval()

all_start_preds = []
all_end_preds = []
all_start_labels = []
all_end_labels = []

for batch in tqdm(validation_dataloader, desc="Evaluating"):
    with torch.no_grad():
        input_ids = batch['input_ids'].squeeze(1).to(device)
        attention_mask = batch['attention_mask'].squeeze(1).to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        start_logits = outputs.start_logits
        end_logits = outputs.end_logits

        start_preds = torch.argmax(start_logits, dim=1).cpu().numpy()
        end_preds = torch.argmax(end_logits, dim=1).cpu().numpy()

        all_start_preds.extend(start_preds)
        all_end_preds.extend(end_preds)
        all_start_labels.extend(start_positions.cpu().numpy())
        all_end_labels.extend(end_positions.cpu().numpy())

metrics = compute_metrics((all_start_preds, all_end_preds), (all_start_labels, all_end_labels))
print(metrics)


Evaluating: 100%|██████████| 500/500 [02:56<00:00,  2.83it/s]

{'start_accuracy': 0.746, 'end_accuracy': 0.7905}
